# NCU輔助選課程式，提供選課時的分發機率參考

## 使用方法
1. 拷貝這份程式到電腦或雲端硬碟(推薦)，以保存選課資料
2. 查找課堂的流水號在**選課系統**->**課程加退選**->**選課**->**志願序與選修別**，可搭配ncu course finder使用
3. 在下方各類的`course_numbers`中輸入你想要的課程流水號，就可查詢該課程的累積人數和分發機率(第1志願有多少機率會上、第2志願有多少機率會上...)。
4. 執行全部程式
5. 根據各個課堂上的分發機率和累積人數，排課程的志願序 EX：已知A堂課**小概率**只有在第1志願才會上，就不用把第2志願的名額給A堂課，可以給有可能在第2志願上的B堂課

## 分發順序
興趣選項課程分發規則如下：
1. 課程修課對象優先順序條件限制
2. 學生登錄之同一課群志願序(此程式算的是這個機率)
3. 系統自動產生之亂數籤號

## 範例結果
- **課程名稱**: 日文(一)A LN0025 - A
- **授課教師**: 王中成 / Wang, Chung-Chen
- **課堂時間**: 星期一 2/3/4
- **人數限制**: 50
- **中選人數**: 49
- **第1志願累積人數**: 172, **機率**: 0.58%

## 注意事項
1. 此程式只適用興趣選項課程(通識、體育、語言類課程等等)，且此程式預設每個人的優先順序都是相同，請先查看你的優先順序是否為第一
2. 執行太多次程式或輸入的課堂號過多可能會鎖IP位置，推薦在colab上執行且課堂號不要一次輸入過多
3. 顯示的分發機率僅供參考，可能會因課程網站的 bug 或優先條件等因素影響實際分發結果。即使程式顯示第一志願的機率為 70%，實際上可能是第一志願 100% 和第二志願 20%。如果某課程的第一志願機率很高，可以考慮將其設為第二志願。

## 製作人員
資工二A 陳聖勳

## 其他問題
請聯繫 email:tom1030507@gmail.com

In [ ]:
!pip install requests
!pip install beautifulsoup4

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

def get_course_detail(course_serial_number):
    # 定義目標URL
    url = 'https://cis.ncu.edu.tw/Course/main/support/courseDetail.html?crs=' + course_serial_number

    # 設置請求頭
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Accept-Language': 'zh-TW,zh;q=0.9'
    }

    # 發送HTTP請求
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    def get_value_by_label(label):
        td = soup.find('td', class_='subTitle', string=label)
        if td:
            next_td = td.find_next_sibling('td')
            if next_td:
                # 排除不需要的部分
                for span in next_td.find_all('span', class_='engclass'):
                    span.decompose()
                return next_td.get_text(strip=True)
        return None

    course_name = get_value_by_label('課程名稱/備註')
    course_code = get_value_by_label('流水號  / 課號')
    course_time = get_value_by_label('時間/教室')
    course_teacher = get_value_by_label('授課教師')

    # 提取課號
    match = re.search(r'[A-Za-z0-9]+ - [A-Za-z*]', course_code)
    if match:
        course_code = match.group()
    else:
        print('未知課號\n')
        return

    enrollment_limit = get_value_by_label('人數限制')

    # 查找特定內容出現的次數
    td_elements = soup.find_all('td')

    selected_count = 0 # 中選人數
    for index in range(len(td_elements) - 1):
        text = td_elements[index].get_text(strip=True)
        if text == '中選' or text == '中選(初選)':
            selected_count += 1

    # 處理時間格式
    if course_time:
        time_matches = re.findall(r'星期[一二三四五六日]\s*\w+', course_time)
        time_slots = {}
        for match in time_matches:
            day, period = re.split(r'\s+', match.strip())
            if day in time_slots:
                time_slots[day].append(period)
            else:
                time_slots[day] = [period]

        course_time = ' / '.join([f"{day} {'/'.join(periods)}" for day, periods in time_slots.items()])

    print(f'課程名稱: {course_name} {course_code}')
    print(f'授課教師: {course_teacher}')
    print(f'課堂時間: {course_time}')
    print(f'人數限制: {enrollment_limit}')
    print(f'中選人數: {selected_count}')

    total_remaining_slots = int(enrollment_limit) - int(selected_count)
    remaining_slots = total_remaining_slots
    accumulated_count = 0 # 累積人數

    for priority in range(1, 10):
        count = 0
        for index in range(len(td_elements) - 1):
            if td_elements[index].get_text(strip=True) == str(priority) and td_elements[index + 1].get_text(strip=True) == '待分發':
                count += 1
        accumulated_count += count
        if count == 0 and remaining_slots > 0:
            print(f'第{priority}志願累積人數: {accumulated_count}, 機率: 100.00%')
            continue
        probability = remaining_slots / count * 100
        if probability > 100:
            probability = 100
        elif probability < 0:
            probability = 0
        print(f'第{priority}志願累積人數: {accumulated_count}, 機率: {probability:.2f}%')
        remaining_slots -= count
        if accumulated_count >= total_remaining_slots * 2:
            break
    print()

# 輸入你想要的課程流水號(4或5位號碼)
pe_course_numbers = ["1041", "1043"]
japanese_course_numbers = ["0001", "0002"]
general_course_numbers = ["9007", "9018"]

print("<體育課程>\n")
for serial_number in pe_course_numbers:
    get_course_detail(serial_number)

print("<日文課程>\n")
for serial_number in japanese_course_numbers:
    get_course_detail(serial_number)

print("<通識課程>\n")
for serial_number in general_course_numbers:
    get_course_detail(serial_number)

<體育課程>

課程名稱: 羽球入門 PE2101 - A
授課教師: 黃相瑋 / Huang, Ching-Wei
課堂時間: 星期二 7/8
人數限制: 40
中選人數: 35
第1志願累積人數: 107, 機率: 4.67%

課程名稱: 羽球入門 PE2101 - C
授課教師: 張哲千 / Chang, Che-Chien
課堂時間: 星期四 5/6
人數限制: 40
中選人數: 39
第1志願累積人數: 71, 機率: 1.41%

<日文課程>

課程名稱: 日文(一)A LN0025 - A
授課教師: 王中成 / Wang, Chung-Chen
課堂時間: 星期一 2/3/4
人數限制: 50
中選人數: 49
第1志願累積人數: 172, 機率: 0.58%

課程名稱: 日文(一)A LN0025 - B
授課教師: 張恆如 / Chang, Herng-Ru
課堂時間: 星期一 A/B/C
人數限制: 50
中選人數: 48
第1志願累積人數: 280, 機率: 0.71%

<通識課程>

課程名稱: 西方古典音樂欣賞 GS2153 - *
授課教師: 翁立美 / Weng,Li-Mei
課堂時間: 星期二 9/A
人數限制: 120
中選人數: 113
第1志願累積人數: 124, 機率: 5.65%

課程名稱: 行政法 GS3077 - *
授課教師: 陳英鈐 / Chen, In-Chin
課堂時間: 星期四 6/7/8
人數限制: 130
中選人數: 116
第1志願累積人數: 19, 機率: 73.68%
第2志願累積人數: 20, 機率: 0.00%
第3志願累積人數: 27, 機率: 0.00%
第4志願累積人數: 32, 機率: 0.00%



#更新日誌
##**2025/2/13**
1. 修正爬取問題：解決了課程網站無法爬取的問題，通過添加 HTTP headers 來實現。
2. 顯示課程時間與授課教師：現在可以顯示課程的具體時間和授課老師，方便學生查看和安排課程。
3. 更新志願序邏輯：發現課程網站的志願序有可能並非最終志願序，因此添加了累積人數並修改了程式邏輯，以便更準確地進行選課。